**_Opening a New Kebap Restaurant in Milano, Italy_**
- Build a dataframe of neighborhoods in Milano, from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a Kebap Restaurant
***
### 1. Import libraries

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
from bs4 import BeautifulSoup 
from sklearn.cluster import KMeans
import json # handle JSON files
from geopy.geocoders import Nominatim # convert to latitude and longitude values
import geocoder 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium 

### 2. Scrap data from Wikipedia page into a DataFrame

In [2]:
# Getting information about Milan
data = requests.get("https://en.wikipedia.org/wiki/Category:Districts_of_Milan").text

In [3]:
# creating beautifulsoup object from html
soup = BeautifulSoup(data, 'html.parser')

In [4]:
soup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Category:Districts of Milan - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"YC7am-@u5NK35TSlMfm1CgAAAJY","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Districts_of_Milan","wgTitle":"Districts of Milan","wgCurRevisionId":899922759,"wgRevisionId":899922759,"wgArticleId":19192884,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Commons category link is on Wikidata","Subdivisions of Milan","City districts"],"wgPageContentLanguage":"en","wgPageContentModel":"wik

In [11]:
neighborhoodList = []

In [12]:
# append neighborhood data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [13]:
# create a new pandas DataFrame from neighborhoods
mi_df = pd.DataFrame({"Neighborhood": neighborhoodList})

mi_df.head()

,Neighborhood
0,Affori
1,Assiano
2,Baggio (district of Milan)
3,Barona
4,Bicocca (district of Milan)


### 3. Get the geographical coordinates

In [15]:
# Function to obtain the locations of the neighborhood in Milan.
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Milan, Italy'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [16]:
# call the previous function to get the coordinates, store in a coords list
coords = [ get_latlng(neighborhood) for neighborhood in mi_df["Neighborhood"].tolist() ]

In [17]:
coords

[[45.51410000000004, 9.173530000000028],
 [45.45061655145966, 9.061632052538704],
 [45.46324000000004, 9.092700000000036],
 [45.433710000000076, 9.15160000000003],
 [45.52149000000003, 9.213260000000048],
 [45.503130000000056, 9.161220000000071],
 [45.515550000000076, 9.150940000000048],
 [45.47470005107049, 9.190009970569658],
 [45.52825000000007, 9.180710000000033],
 [45.456180000000074, 9.224880000000041],
 [45.501986915580225, 9.264641435209318],
 [45.41719000000006, 9.23971000000006],
 [45.500860000000046, 9.265130000000056],
 [45.503460000000075, 9.248800000000074],
 [45.47708000000006, 9.226600000000076],
 [45.52631000000008, 9.158870000000036],
 [45.3841130823799, 9.13262826573227],
 [45.51054000000005, 9.24386000000004],
 [45.504110000000026, 9.176470000000052],
 [45.49234000000007, 9.078520000000026],
 [45.45975000000004, 9.24690000000004],
 [45.496710000000064, 9.114840000000072],
 [45.50469000000004, 9.136970000000076],
 [45.49631000000005, 9.169400000000053],
 [45.44466237

In [18]:
# create temporary dataframe
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [19]:
mi_df['Latitude'] = df_coords['Latitude']
mi_df['Longitude'] = df_coords['Longitude']

In [20]:
# check the neighborhoods and the coordinates in main dataframe
print(mi_df.shape)
mi_df

(77, 3)


,Neighborhood,Latitude,Longitude
0,Affori,45.514100,9.173530
1,Assiano,45.450617,9.061632
2,Baggio (district of Milan),45.463240,9.092700
3,Barona,45.433710,9.151600
4,Bicocca (district of Milan),45.521490,9.213260
5,Bovisa,45.503130,9.161220
6,Bovisasca,45.515550,9.150940
7,Brera (district of Milan),45.474700,9.190010
8,Bruzzano,45.528250,9.180710
9,Calvairate,45.456180,9.224880


In [21]:
# save it
mi_df.to_csv("mi_df.csv", index=False)

### 4. Create a map of Milano with neighborhoods superimposed on top

In [22]:
# get the coordinates of Milano
address = 'Milan, Italy'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Milan,Italy {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Milan,Italy 45.4668, 9.1905.


In [23]:
# create a map
map_mi = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers 
for lat, lng, neighborhood in zip(mi_df['Latitude'], mi_df['Longitude'], mi_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mi)  
    
map_mi

In [24]:
# save it as html file
map_mi.save('map_mi.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [27]:
# I deleted ID and Client_secret nums here because they contain my own information,
# enter information numbers of your foursquare developer account to try.

CLIENT_ID = 'xXx'
CLIENT_SECRET = 'xXx'
VERSION = 'xXx'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HRATK3DWPMTCRK05GTAMH0C4VRZYRQQPWBQP4OGS42YCOOOT
CLIENT_SECRET:SE2DYZ1Z55JNY1ZN3RL00Q0CVBTO24IHH0XMBD0XTLOYCYOW


**Now, let's see the top 200 venues that are within a radius of 2000 meters.**

In [28]:
radius = 2000
LIMIT = 200

venues = []

for lat, long, neighborhood in zip(mi_df['Latitude'], mi_df['Longitude'], mi_df['Neighborhood']):
    
    # create API request 
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [29]:
# convert venues list to a new DataFrame (venues_df)
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(5209, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Affori,45.5141,9.17353,Al Paradiso Della Pizza,45.511351,9.175416,Pizza Place
1,Affori,45.5141,9.17353,Esselunga,45.512380,9.173461,Supermarket
2,Affori,45.5141,9.17353,Biologic Bar & Restaurant,45.512302,9.178709,Hotel Bar
3,Affori,45.5141,9.17353,Effecorta,45.517989,9.169883,Health Food Store
4,Affori,45.5141,9.17353,Osteria del biliardo,45.514741,9.169345,Pool Hall


> **Let's check how many venues are returned for each neighborhood**

In [39]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Affori,69,69,69,69,69,69
Assiano,10,10,10,10,10,10
Baggio (district of Milan),20,20,20,20,20,20
Barona,51,51,51,51,51,51
Bicocca (district of Milan),100,100,100,100,100,100
Bovisa,100,100,100,100,100,100
Bovisasca,44,44,44,44,44,44
Brera (district of Milan),100,100,100,100,100,100
Bruzzano,35,35,35,35,35,35


**How many unique categories are there from the places mentioned to us, let's find it.**

In [40]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 287 uniques categories.


In [41]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Pizza Place', 'Supermarket', 'Hotel Bar', 'Health Food Store',
       'Pool Hall', 'Kebab Restaurant', 'Park', 'Hotel', 'Brewery',
       'Ballroom', 'Italian Restaurant', 'Soccer Field', 'Hobby Shop',
       'Paper / Office Supplies Store', 'Theater', 'Art Gallery',
       'Ice Cream Shop', 'Food', 'Steakhouse', 'Fried Chicken Joint',
       'Café', 'Bar', 'Plaza', 'Peruvian Restaurant', 'Music Store',
       'Pub', 'Vegetarian / Vegan Restaurant', 'Bakery',
       'Gym / Fitness Center', 'Cocktail Bar', 'Diner',
       'General Entertainment', 'Food Court', 'Tram Station', 'Bookstore',
       'Restaurant', 'Hostel', 'Piadineria', 'Rock Club',
       'Convenience Store', 'Football Stadium', 'Bus Stop', 'Cafeteria',
       'Market', 'Mediterranean Restaurant', 'Neighborhood', 'Juice Bar',
       'Airport', 'Athletics & Sports', 'Snack Place'], dtype=object)

In [42]:
# check if the results contain "Kebap Restaurant"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [43]:
# one hot encoding
mi_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mi_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mi_onehot.columns[-1]] + list(mi_onehot.columns[:-1])
mi_onehot = mi_onehot[fixed_columns]

print(mi_onehot.shape)
mi_onehot.head()

(5209, 288)


,Neighborhoods,Abruzzo Restaurant,Accessories Store,Adult Education Center,African Restaurant,Agriturismo,Airport,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Ballroom,Bar,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Calabria Restaurant,Camera Store,Campanian Restaurant,Campground,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cave,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comedy Club,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Doner Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,Hobby Shop,Hockey Arena,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lombard Restaurant,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Palace,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Piadineria,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Puglia Restaurant,Racecourse,Racetrack,Radio Station,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Rock Club,Roman Restaurant,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sardinian Restaurant,Sauna / Steam Room,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Sicilian Restaurant,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Tyrolean Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toll Plaza,Toy / Game Store,Track,Trail,Train Station,Tram Station,Trattoria/Osteria,Tunnel,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse S

**Next, let's group the frequencies by neighborhood and category.**

In [44]:
mi_grouped = mi_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mi_grouped.shape)
mi_grouped

(77, 288)


,Neighborhoods,Abruzzo Restaurant,Accessories Store,Adult Education Center,African Restaurant,Agriturismo,Airport,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Ballroom,Bar,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Calabria Restaurant,Camera Store,Campanian Restaurant,Campground,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cave,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comedy Club,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Doner Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,Hobby Shop,Hockey Arena,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lombard Restaurant,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Palace,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Piadineria,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Puglia Restaurant,Racecourse,Racetrack,Radio Station,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Rock Club,Roman Restaurant,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sardinian Restaurant,Sauna / Steam Room,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Sicilian Restaurant,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Tyrolean Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toll Plaza,Toy / Game Store,Track,Trail,Train Station,Tram Station,Trattoria/Osteria,Tunnel,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse S

In [45]:
# Let's find the ones in  Kebap Restaurant
len(mi_grouped[mi_grouped["Kebab Restaurant"] > 0])

12

**Create a new DataFrame for Kebap Restaurant data only**

In [58]:
mi_kebap = mi_grouped[["Neighborhoods","Kebab Restaurant"]]

In [59]:
mi_kebap.head()

,Neighborhoods,Kebab Restaurant
0,Affori,0.028986
1,Assiano,0.000000
2,Baggio (district of Milan),0.000000
3,Barona,0.000000
4,Bicocca (district of Milan),0.010000


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Milano into 3 clusters.

In [60]:
# set cluster number = 2
kclusters = 3

mi_clustering = mi_kebap.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mi_clustering)

kmeans.labels_[0:10] 

array([1, 0, 0, 0, 2, 1, 0, 0, 0, 0])

In [61]:
# create a new data frame that includes the cluster and the 10 locations for each neighborhood.
mi_merged = mi_kebap.copy()

mi_merged["Cluster Labels"] = kmeans.labels_

In [62]:
mi_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mi_merged.head()

,Neighborhood,Kebab Restaurant,Cluster Labels
0,Affori,0.028986,1
1,Assiano,0.000000,0
2,Baggio (district of Milan),0.000000,0
3,Barona,0.000000,0
4,Bicocca (district of Milan),0.010000,2


In [63]:
mi_merged = mi_merged.join(mi_df.set_index("Neighborhood"), on="Neighborhood")

print(mi_merged.shape)
mi_merged.head() 

(77, 5)


,Neighborhood,Kebab Restaurant,Cluster Labels,Latitude,Longitude
0,Affori,0.028986,1,45.514100,9.173530
1,Assiano,0.000000,0,45.450617,9.061632
2,Baggio (district of Milan),0.000000,0,45.463240,9.092700
3,Barona,0.000000,0,45.433710,9.151600
4,Bicocca (district of Milan),0.010000,2,45.521490,9.213260


In [64]:
print(mi_merged.shape)
mi_merged.sort_values(["Cluster Labels"], inplace=True)
mi_merged

(77, 5)


,Neighborhood,Kebab Restaurant,Cluster Labels,Latitude,Longitude
38,Porta Garibaldi (Milan),0.000000,0,45.480650,9.187310
75,Vigentino,0.000000,0,45.433720,9.201040
39,Porta Genova,0.000000,0,45.457900,9.174570
40,Porta Lodovica,0.000000,0,45.453180,9.189290
41,Porta Magenta,0.000000,0,45.444451,8.886532
42,Porta Monforte,0.000000,0,45.467233,9.202522
43,Porta Nuova (Milan),0.000000,0,45.479710,9.192470
44,Porta Romana (Milan),0.000000,0,45.456900,9.200950
45,Porta Sempione,0.000000,0,45.506105,9.100737
46,Porta Tenaglia,0.000000,0,45.477664,9.182243


**Finally, let's visualize the resulting clusters**

In [65]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers
markers_colors = []
for lat, lon, poi, cluster in zip(mi_merged['Latitude'], mi_merged['Longitude'], mi_merged['Neighborhood'], mi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [66]:
# save the map
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [67]:
mi_merged.loc[mi_merged['Cluster Labels'] == 0]

,Neighborhood,Kebab Restaurant,Cluster Labels,Latitude,Longitude
38,Porta Garibaldi (Milan),0.0,0,45.480650,9.187310
75,Vigentino,0.0,0,45.433720,9.201040
39,Porta Genova,0.0,0,45.457900,9.174570
40,Porta Lodovica,0.0,0,45.453180,9.189290
41,Porta Magenta,0.0,0,45.444451,8.886532
42,Porta Monforte,0.0,0,45.467233,9.202522
43,Porta Nuova (Milan),0.0,0,45.479710,9.192470
44,Porta Romana (Milan),0.0,0,45.456900,9.200950
45,Porta Sempione,0.0,0,45.506105,9.100737
46,Porta Tenaglia,0.0,0,45.477664,9.182243



#### Cluster 1

In [68]:
mi_merged.loc[mi_merged['Cluster Labels'] == 1]

,Neighborhood,Kebab Restaurant,Cluster Labels,Latitude,Longitude
5,Bovisa,0.030000,1,45.50313,9.16122
0,Affori,0.028986,1,45.51410,9.17353
18,Dergano,0.020000,1,45.50411,9.17647
66,San Cristoforo sul Naviglio (district of Milan),0.023810,1,45.44763,9.15458


#### Cluster 2

In [69]:
mi_merged.loc[mi_merged['Cluster Labels'] == 2] 

,Neighborhood,Kebab Restaurant,Cluster Labels,Latitude,Longitude
57,Quartiere Feltre,0.010000,2,45.491379,9.244672
68,Segnano,0.010000,2,45.506910,9.206050
54,Precotto,0.010000,2,45.515410,9.225530
53,Prato Centenaro,0.010000,2,45.506710,9.199210
4,Bicocca (district of Milan),0.010000,2,45.521490,9.213260
52,Portello (district of Milan),0.010870,2,45.474760,9.146120
25,Gorla,0.010000,2,45.505900,9.222640
35,Nosedo,0.012048,2,45.433810,9.221370


#### Observations:
Most of the kebab restaurants are concentrated in areas that are slightly away from the center of the city of Milan, with the highest rate in cluster 1 and moderate rate in cluster 2.
   
        
This represents a great opportunity and high potential to open new kebab restaurants as there is little or no competition from existing kebab restaurants. Meanwhile, kebab restaurants in the neighborhoods in cluster 1 suffer from intense competition due to the excess supply and high kebab restaurant density. From another point of view, this suggests that excessive supply of kebab restaurants mostly takes place in the central area of ​​the city, still having very few kebab restaurants in the suburban areas and the city center. Therefore, this project advises property developers to use these findings to open a new kebab restaurant in the neighborhoods of cluster 0.
        
       
Property developers, with unique sales proposals to stand out in the competition, can also open new kebab restaurants in neighborhoods in cluster 2 with moderate competition. Finally, property developers are advised to avoid neighborhoods in cluster 1, which is already a highly concentrated kebab restaurant and suffers from intense competition.